In [1]:
import numpy as np
import random

numpy digunakan untuk operasi array dan perhitungan numerik.
random digunakan untuk pengacakan, seperti membuat posisi acak pada papan permainan.

In [10]:
board_size = 4
num_pairs = 8
total_states = board_size * board_size
episodes = 200

board_size: Ukuran papan (4x4).
num_pairs: Jumlah pasangan angka yang harus dicocokkan.
total_states: Total jumlah posisi di papan (4×4=164×4=16).
episodes: Jumlah iterasi pelatihan untuk algoritme Q-Learning.

In [11]:
learning_rate = 0.1
discount_factor = 0.9
epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.01

learning_rate: Kecepatan pembelajaran untuk memperbarui nilai Q-Table.
discount_factor: Seberapa jauh algoritme mempertimbangkan reward masa depan.
epsilon: Parameter untuk eksplorasi, yang akan menurun seiring waktu.
epsilon_decay: Faktor penurunan epsilon di setiap episode.
min_epsilon: Batas minimal nilai epsilon.

In [12]:
def create_board():
    pairs = list(range(1, num_pairs + 1)) * 2
    random.shuffle(pairs)
    return np.array(pairs).reshape(board_size, board_size)

Membuat papan permainan dengan pasangan angka acak.
Pasangan angka digandakan (karena setiap angka harus dicocokkan) dan diacak menggunakan random.shuffle.
Dikembalikan sebagai array 2D dengan ukuran sesuai board_size.

In [13]:
def get_reward(board, pos1, pos2):
    if board[pos1] == board[pos2]:
        return 10
    else:
        return -1

Menghitung reward berdasarkan pasangan yang dipilih:
    Jika angka di posisi pos1 dan pos2 cocok, reward = 10.
    Jika tidak cocok, reward = -1.

In [14]:
def choose_action(state, q_table, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, total_states - 1), random.randint(0, total_states - 1)
    else:
        return divmod(np.argmax(q_table[state]), board_size)

Fungsi ini memilih dua posisi di papan permainan berdasarkan strategi exploration-exploitation:
    Jika bilangan acak antara 0 dan 1 lebih kecil dari nilai epsilon, eksplorasi dilakukan dengan memilih dua posisi secara acak.
    Jika tidak, eksploitasi dilakukan dengan memilih aksi berdasarkan nilai maksimum dari Q-Table untuk keadaan tertentu

In [15]:
q_table = np.zeros((total_states, total_states))

Q-Table diinisialisasi sebagai matriks nol dengan ukuran . Matriks ini digunakan untuk menyimpan nilai Q untuk setiap pasangan aksi (dua posisi di papan).

In [16]:
for episode in range(episodes):
    board = create_board()
    revealed = np.zeros_like(board)
    state = random.randint(0, total_states - 1)
    total_reward = 0

    while not np.all(revealed):
        pos1, pos2 = choose_action(state, q_table, epsilon)

        while pos1 == pos2:
            pos1, pos2 = choose_action(state, q_table, epsilon)

        reward = get_reward(board, np.unravel_index(pos1, (board_size, board_size)),
                            np.unravel_index(pos2, (board_size, board_size)))

        new_state = random.randint(0, total_states - 1)
        q_table[state, pos1] += learning_rate * (
            reward + discount_factor * np.max(q_table[new_state]) - q_table[state, pos1]
        )
        q_table[state, pos2] += learning_rate * (
            reward + discount_factor * np.max(q_table[new_state]) - q_table[state, pos2]
        )

        if reward == 10:
            revealed[np.unravel_index(pos1, (board_size, board_size))] = 1
            revealed[np.unravel_index(pos2, (board_size, board_size))] = 1

        total_reward += reward
        state = new_state

    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    if episode % 20 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}")

print("Training selesai.")

Episode 0, Total Reward: -131
Episode 20, Total Reward: 13
Episode 40, Total Reward: -177
Episode 60, Total Reward: -262
Episode 80, Total Reward: -332
Episode 100, Total Reward: -28
Episode 120, Total Reward: -113
Episode 140, Total Reward: -396
Episode 160, Total Reward: -177
Episode 180, Total Reward: -528
Training selesai.


Setiap episode pelatihan dimulai dengan:
    Membuat papan permainan baru menggunakan fungsi create_board.
    Menginisialisasi array revealed untuk melacak posisi yang sudah ditemukan pasangannya.
    Menentukan keadaan awal secara acak.
    Mengatur ulang total_reward untuk menyimpan reward yang diterima selama satu episode.

Selama semua pasangan di papan belum ditemukan (np.all(revealed) bernilai False), loop ini akan terus berjalan.
Dua posisi (pos1pos1 dan pos2pos2) dipilih menggunakan fungsi choose_action. Jika kedua posisi sama, posisi baru dipilih kembali.

Reward dihitung menggunakan fungsi get_reward, berdasarkan nilai di dua posisi yang dipilih.
Keadaan baru dipilih secara acak.
Nilai Q di Q-Table diperbarui untuk kedua posisi yang dipilih menggunakan rumus Q-Learning:
Q(s,a)=Q(s,a)+α[r+γmax⁡aQ(s′,a)−Q(s,a)]
Q(s,a)=Q(s,a)+α[r+γamax​Q(s′,a)−Q(s,a)]

    s: Keadaan saat ini.
    a: Aksi yang dipilih (pos1 atau pos2).
    r: Reward yang diterima.
    s′: Keadaan berikutnya.
    α: Learning rate.
    γ: Discount factor.
Jika pasangan cocok (reward = 10), posisi tersebut ditandai sebagai sudah ditemukan di array revealed.
Nilai epsilon dikurangi di setiap episode menggunakan faktor epsilon_decay. Hal ini mengurangi frekuensi eksplorasi seiring dengan berjalannya pelatihan, sehingga model semakin fokus pada eksploitasi.
Setiap 20 episode, total reward untuk episode tersebut dicetak untuk memantau kemajuan pelatihan.
Setelah semua episode selesai, program mencetak "Training selesai."
    